In [16]:
import numpy
import cv2
import heapq
import statistics

In [31]:
def popularity(image,k):
        (m,n,_) = image.shape
        d = {}
        for i in range(m):
            for j in range(n):
                t = tuple(image[i,j])
                if t in d:
                    d[t] += 1
                else:
                    d[t] = 1
        top_k_colors =heapq.nlargest(k, d, key=d.get)
        return top_k_colors

In [23]:
def split(list_of_tuples,colors,color_to_split,k):
    if (k==1):
        return [sum([x[0]*x[1] for x in list_of_tuples])/(sum([x[1] for x in list_of_tuples]))]
    median = statistics.median([x[0][color_to_split] for x in list_of_tuples])
    left = filter(lambda x: x[0][color_to_split] <= median , list_of_tuples)
    right = filter(lambda x: x[0][color_to_split] > median, list_of_tuples)
    return (split(left,colors,(color_to_split+1)%colors,k/2)+split(right,colors,(color_to_split+1)%colors,k/2))

In [49]:
def median_cut(image,k):
    (m,n,c) = image.shape
    d = {}
    for i in range(m):
        for j in range(n):
            t = tuple(image[i,j])
            if t in d:
                d[t] += 1
            else:
                d[t] = 1
    l = d.values()
    print(l)
    # color to split missing
    split(l,c,k)

In [50]:
# test_image = cv2.imread('test1.png', cv2.IMREAD_GRAYSCALE)
test_image = cv2.imread('test1.png')
# cv2.imshow('test_image',test_image)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [51]:
print(tuple(test_image[0,0]))

(246, 246, 243)


In [52]:
median_cut(test_image, 16)

dict_values([277, 20792, 6968, 126, 68, 30, 225, 135, 190, 268, 83, 525, 781, 158, 137, 1075, 1328, 1492, 1726, 1651, 1484, 193, 193, 1366, 1292, 107, 123, 1268, 1200, 1015, 941, 46, 594, 2, 37, 35, 41, 55, 177, 5, 6, 15, 180, 46, 54, 19, 1, 57, 30, 139, 242, 2, 4, 11, 27, 5, 2, 3, 124, 239, 11, 109, 83, 58, 173, 211, 49, 46, 13, 42, 12, 24, 10, 80, 8, 6, 4, 1, 6, 7, 6, 24, 13, 4, 14, 7, 28, 27, 8, 31, 24, 45, 21, 4, 8, 67, 62, 10, 17, 6, 2, 3, 3, 4, 1, 2, 4, 3, 2, 1, 1, 3, 6, 5, 7, 8, 14, 15, 3, 1, 1, 5, 5, 99, 21, 20, 13, 10, 2, 5, 1, 2, 3, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3, 1, 2, 3, 3, 2, 1, 4, 1, 1, 1, 1, 2, 6, 7, 11, 2, 12, 20, 89, 74, 71, 62, 22, 9, 3, 3, 1, 3, 4, 4, 5, 4, 3, 4, 1, 2, 2, 1, 3, 1, 1, 1, 3, 1, 2, 1, 2, 3, 4, 3, 6, 4, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 3, 18, 21, 25, 4, 15, 9, 11, 5, 5, 7, 1, 2, 3, 2, 1, 1, 3, 1, 1, 3, 2, 1, 1, 2, 8, 3, 1, 3, 2, 3, 5, 6, 1, 3, 2, 2, 6, 5, 3, 6, 2, 2, 2, 5, 3, 3, 6, 2, 6, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 4, 9, 4, 6, 27, 16, 1

TypeError: split() missing 1 required positional argument: 'k'

In [33]:
plist = popularity(test_image, 4)

In [34]:
print(plist)

[(233, 233, 227), (232, 232, 226), (243, 243, 237), (244, 244, 238)]


In [6]:
a = [1,2,3,4]
filter(lambda x: 

SyntaxError: unexpected EOF while parsing (<ipython-input-6-a565004daea9>, line 2)

In [7]:
5%2

1

In [8]:
help(filter)

Help on class filter in module builtins:

class filter(object)
 |  filter(function or None, iterable) --> filter object
 |  
 |  Return an iterator yielding those items of iterable for which function(item)
 |  is true. If function is None, return the items that are true.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.

